In [7]:
import pandas as pd
import pandas_datareader.data as pdr
import datetime as dt
import math
import yfinance as yf
import matplotlib.pyplot as plt

yf.pdr_override()

In [3]:
def cagr(_df):
    df = _df.copy()
    df = df.sort_index(ascending=True)
    df["cum_return"] = (1 + df["return"]).cumprod()
    total_ret = df['cum_return'][-1]
    start = datetime.utcfromtimestamp(df.index.values[0].astype('O')/1e9)
    end = datetime.utcfromtimestamp(df.index.values[-1].astype('O')/1e9)
    period_years = (end - start).days / 365.25
    CAGR = (total_ret)**(1/period_years) - 1
    return CAGR

In [4]:
def volatility(_df):
    start = datetime.utcfromtimestamp(_df.index.values[0].astype('O')/1e9)
    end = datetime.utcfromtimestamp(_df.index.values[-1].astype('O')/1e9)
    period_years = (end - start).days / 365.25
    ratio_to_annual = _df['return'].count() / period_years
    vol = _df["return"].std() * math.sqrt(ratio_to_annual)
    return vol

In [5]:
def sharpe(_df, risk_free_rate):
    ret = cagr(_df)
    vol = volatility(_df)
    sharpe = (ret - risk_free_rate) / vol
    return sharpe

In [6]:
def max_drawdown(_df):
    df = _df.copy()
    df = df.sort_index(ascending=True)
    df["cum_return"] = (1 + df["return"]).cumprod()
    df["cum_return_max"] = df["cum_return"].cummax()
    df["drawdown"] = df["cum_return_max"] - df["cum_return"]
    df["drawdown_pct"] = df["drawdown"] / df["cum_return_max"]
    max_dd = df.loc[df["drawdown_pct"].idxmax()]
    return {
        'drawdown': max_dd['drawdown_pct'],
        'date': max_dd.name
    }